In [1]:
import transformers
import tensorflow
import keras
import torch

print(transformers.__version__)
print(tensorflow.__version__)
print(keras.__version__)
print(torch.__version__)

c:\Users\victo\anaconda3\envs\artofproblemsolving\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4.35.2
2.10.1
2.10.0
2.2.0


In [31]:
# Import library needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DataCollatorWithPadding
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from torch.utils.data import TensorDataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

In [32]:
# Load dataset
df = pd.read_csv("imo.csv")
# Select not null data
df_notna = df[df["label"].notna()]
# Reset index
df_notna.reset_index(drop=True, inplace=True)

# DISPLAY INFORMATION DATASET
algebra = df_notna[df_notna['label'] == 'Algebra'].shape[0]
combinatorics = df_notna[df_notna['label'] == 'Combinatorics'].shape[0]
geometry = df_notna[df_notna['label'] == 'Geometry'].shape[0]
number_theory = df_notna[df_notna['label'] == 'Number Theory'].shape[0]

print(f"Algebra       = {algebra}")
print(f"Combinatorics = {combinatorics}")
print(f"Geometry      = {geometry}")
print(f"Number Theory = {number_theory}")


Algebra       = 683
Combinatorics = 565
Geometry      = 962
Number Theory = 661


In [33]:
# Replace label with integer values (encode)
display(df_notna[["id_key", "no", "year", "post_rendered", "post_canonical", "label"]])

le = LabelEncoder()
df_notna['label_encoded'] = le.fit_transform(df_notna['label'])

display(df_notna[["id_key", "no", "year", "post_rendered", "post_canonical", "label", "label_encoded"]])


,id_key,no,year,post_rendered,post_canonical,label
0,1,1,2023imo,"Determine all composite integers <img src=""//l...",Determine all composite integers $n>1$ that sa...,Number Theory
1,2,2,2023imo,"Let <img src=""//latex.artofproblemsolving.com/...",Let $ABC$ be an acute-angled triangle with $AB...,Geometry
2,3,3,2023imo,"For each integer <span style=""white-space:pre;...","For each integer $k \geqslant 2$, determine al...",Number Theory
3,4,4,2023imo,"Let <img src=""//latex.artofproblemsolving.com/...","Let $x_1,x_2,\dots,x_{2023}$ be pairwise diffe...",Algebra
4,5,5,2023imo,"Let <img src=""//latex.artofproblemsolving.com/...",Let $n$ be a positive integer. A Japanese tria...,Combinatorics
...,...,...,...,...,...,...
2866,10938,2,1997_tuymaada_olympiad,Solve in natural numbers the system of equatio...,Solve in natural numbers the system of equatio...,Number Theory
2867,10941,5,1997_tuymaada_olympiad,"Prove the inequality <img src=""//latex.artofpr...",Prove the inequality $\left(1+\frac{1}{q}\righ...,Algebra
2868,10943,7,1997_tuymaada_olympiad,It is known that every student of the class fo...,It is known that every student of the class fo...,Combinatorics
2869,10955,3,1995_tuymaada_olympiad,"Prove that the equation <img src=""//latex.art...",Prove that the equation $(\sqrt5 +1)^{2x}+ (\s...,Number Theory


C:\Users\victo\AppData\Local\Temp\ipykernel_18072\2941989690.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notna['label_encoded'] = le.fit_transform(df_notna['label'])


,id_key,no,year,post_rendered,post_canonical,label,label_encoded
0,1,1,2023imo,"Determine all composite integers <img src=""//l...",Determine all composite integers $n>1$ that sa...,Number Theory,3
1,2,2,2023imo,"Let <img src=""//latex.artofproblemsolving.com/...",Let $ABC$ be an acute-angled triangle with $AB...,Geometry,2
2,3,3,2023imo,"For each integer <span style=""white-space:pre;...","For each integer $k \geqslant 2$, determine al...",Number Theory,3
3,4,4,2023imo,"Let <img src=""//latex.artofproblemsolving.com/...","Let $x_1,x_2,\dots,x_{2023}$ be pairwise diffe...",Algebra,0
4,5,5,2023imo,"Let <img src=""//latex.artofproblemsolving.com/...",Let $n$ be a positive integer. A Japanese tria...,Combinatorics,1
...,...,...,...,...,...,...,...
2866,10938,2,1997_tuymaada_olympiad,Solve in natural numbers the system of equatio...,Solve in natural numbers the system of equatio...,Number Theory,3
2867,10941,5,1997_tuymaada_olympiad,"Prove the inequality <img src=""//latex.artofpr...",Prove the inequality $\left(1+\frac{1}{q}\righ...,Algebra,0
2868,10943,7,1997_tuymaada_olympiad,It is known that every student of the class fo...,It is known that every student of the class fo...,Combinatorics,1
2869,10955,3,1995_tuymaada_olympiad,"Prove that the equation <img src=""//latex.art...",Prove that the equation $(\sqrt5 +1)^{2x}+ (\s...,Number Theory,3


In [34]:
# Create a dictionary to map encoded labels back to original labels
label_map_rev = {label: i for i, label in enumerate(le.classes_)}
label_map_rev


{'Algebra': 0, 'Combinatorics': 1, 'Geometry': 2, 'Number Theory': 3}

In [35]:
# Preprocess text (optional)
df_notna["post_canonical"] = df_notna["post_canonical"].str.lower()
# Consider other preprocessing steps

# Split X, y dataset
X, y = df_notna["post_canonical"], df_notna["label_encoded"]

# Split train & test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DISPLAY DATA (observe column 'post_canonical') AFTER PREPROCESSED
# display(df_notna[["id_key", "no", "year", "post_rendered", "post_canonical", "label", "label_encoded"]])


C:\Users\victo\AppData\Local\Temp\ipykernel_18072\1720640278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_notna["post_canonical"] = df_notna["post_canonical"].str.lower()


In [36]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize inputs
max_length = 128
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_length)

# Convert to PyTorch tensors
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_masks = torch.tensor(train_encodings['attention_mask'])
train_labels = torch.tensor(y_train.values)

test_input_ids = torch.tensor(test_encodings['input_ids'])
test_attention_masks = torch.tensor(test_encodings['attention_mask'])
test_labels = torch.tensor(y_test.values)


In [37]:
# Create data loaders
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [38]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


c:\Users\victo\anaconda3\envs\artofproblemsolving\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:
# Train the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [41]:
for epoch in range(epochs):
    for batch in train_loader:
        input_ids, attention_masks, labels = batch
        input_ids, attention_masks, labels = input_ids.to(device), attention_masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'

In [5]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=max(df_notna['label'])+1)
y_test = to_categorical(y_test, num_classes=max(df_notna['label'])+1)


In [6]:
import tensorflow_hub as hub
import tensorflow_text as text

In [9]:
# Load BERT and the preprocessing model from TF Hub.
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [ ]:
import tensorflow as tf

In [ ]:
# BERT Layers

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

print(outputs['pooled_output'].shape)

# Neural Network Layers
first = tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
second = tf.keras.layers.Dense(4,activation='softmax', name='output')(first)

# Construct the final model
model = tf.keras.Model(inputs=[text_input], outputs=[second])


In [11]:
torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device(type='cuda')

In [10]:
# Tokenize text
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", padding=True)

# Create dataset classes with padding
class TextDataset(Dataset):
    def __init__(self, text, labels):
        self.text = text
        self.labels = labels

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = self.text[idx]
        label = self.labels[idx]
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        return {
            'input_ids': inputs['input_ids'].squeeze(), 
            'attention_mask': inputs['attention_mask'].squeeze()
            }, label

# Create datasets with padding
train_dataset = TextDataset(X_train.tolist(), y_train.tolist())
test_dataset = TextDataset(X_test.tolist(), y_test.tolist())

# Encode labels (after splitting for consistency)
label_encoder = {label: i for i, label in enumerate(df_notna["label"].unique())}
y_train = y_train.apply(lambda x: label_encoder[x])
y_test = y_test.apply(lambda x: label_encoder[x])

print(label_encoder)
print(y_train)
print(y_test)


{'Number Theory': 0, 'Geometry': 1, 'Algebra': 2, 'Combinatorics': 3}
1423    2
958     0
605     3
2561    3
612     1
       ..
1638    1
1095    1
1130    0
1294    2
860     2
Name: label, Length: 2296, dtype: int64
729     3
2067    1
1288    2
1738    0
2078    0
       ..
2597    1
163     1
1913    2
252     1
1801    0
Name: label, Length: 575, dtype: int64


In [11]:
# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_encoder))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Set up training
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# Define data collator for padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train model
for epoch in range(3):  # Adjust number of epochs
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
# Evaluate model
model.eval()
test_loader = DataLoader(test_dataset, batch_size=16)
predictions = []
true_labels = []
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(**inputs)
        predicted_labels = torch.argmax(outputs.logits, dim=1)
        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

# Print evaluation metrics
print("Accuracy:", accuracy_score(true_labels, predictions))
print("F1 Score:", f1_score(true_labels, predictions, average="weighted"))
print("Precision:", precision_score(true_labels, predictions, average="weighted"))
print("Recall:", recall_score(true_labels, predictions, average="weighted"))
